# A demo how to work with `QAOAParameter` classes

Farhi et al proposed in their original paper about QAOA the following Ansatz for the QAOA circuit:

$$
U = e^{-i \beta_p H_0} e^{-i \gamma_p H_1} \cdots e^{-i \beta_0 H_0} e^{-i \gamma  H_1}.
$$
...

### ToDo for this Demo
 - [ ] Finish the explanation of the different parametrizations
 - [ ] Explain _well_ what the `gammas_pairs`, `gammas_singles` and `betas` are in the raw format.
 - [ ] Demo _all_ different parametrizations that we have

In [16]:
# import the standard modules from python
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters that we want to demo
from qaoa.parameters import AdiabaticTimestepsQAOAParameters, AlternatingOperatorsQAOAParameters, AbstractQAOAParameters, GeneralQAOAParameters, QAOAParameterIterator, FourierQAOAParameters

Let us first create a hamiltonian to work with:

In [2]:
# create a hamiltonian on 3 qubits with 2 coupling terms and 1 bias term
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0


## Building the parameters automatically with `.linear_ramp_from_hamiltonian()`

The easiest way to create `AlternatingOperatorsQAOAParameters` that work exactly with our Hamiltonian, is to use the `AlternatingOperatorsQAOAParameters.linear_ramp_from_hamiltonian` function:

In [4]:
timesteps = 2       # equivalent to p in Farhis original paper
time = 1            # total time of the annelaing schedule
params = AlternatingOperatorsQAOAParameters\
            .linear_ramp_from_hamiltonian(hamiltonian, timesteps, time = time) 
print(params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.375, 0.125]
	gammas_singles: [0.125, 0.375]
	gammas_pairs: [0.125, 0.375]



We see, that if we print `params`, it gives us the timesteps the angles and on which combinations of qubits they act. But we can also have a look at the _internal_ `x_rotation_angles`, `z_rotation_angles` and `zz_rotation_angles` that were automatically calculated under the hood. These are the coefficients that are applied to the single terms in the exponentiated hamiltonian.

In [13]:
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)


 x_rotation_angles:
 [[0.0, 0.0, 0.0], [0.125, 0.125, 0.125]]

 z_rotation_angles:
 [[-0.0625], [-0.1875]]

 zz_rotation_angles:
 [[0.0875, 0.15], [0.26249999999999996, 0.44999999999999996]]


Now we could pass these `params` straight to a cost_function that we built with `qaoa.cost_function.QAOACostFunctionOnQVM` or with `qaoa.cost_function.QAOACostFunctionOnWFSim`. But often we will be interested in modifying the parameters, before running QAOA with them. The way the cost_function does this, is by getting a flat list with parameters via `params.raw()`, changing the values in that list and feeding it back to `params` via `params.update()`:

In [9]:
# get the raw parameters and print them
raw = params.raw()
print("the raw parameters:\n", raw)

# change a value and put it back via params.update()
raw[0] = 0.0
print("\nthe new raw parameters:\n", raw)
params.update_from_raw(raw)
print("\nand the updated parameters:\n", params)

the raw parameters:
 [0.0, 0.125, 0.125, 0.375, 0.125, 0.375]

the new raw parameters:
 [0.0, 0.125, 0.125, 0.375, 0.125, 0.375]

and the updated parameters:
 Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.0, 0.125]
	gammas_singles: [0.125, 0.375]
	gammas_pairs: [0.125, 0.375]



and we can convince us that under the hood the `gammas_singles`, `gammas_pairs` and `betas` were also automatically updated:

In [14]:
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)


 x_rotation_angles:
 [[0.0, 0.0, 0.0], [0.125, 0.125, 0.125]]

 z_rotation_angles:
 [[-0.0625], [-0.1875]]

 zz_rotation_angles:
 [[0.0875, 0.15], [0.26249999999999996, 0.44999999999999996]]


## Building the parameters manually

Sometimes we don't want to build a QAOAParametersObject automaticall via `.linear_ramp_from_hamiltonian()`, but want some more finde grained control. So we can instead specify all that needs to be know manually and passing that to the constructor.
Fully specified QAOAParameters consist of hyperparameters, that don't change over the lifetime of a QAOA optimization run and "normal" parameters that are varied during the optimization.

The hyperparameters are the number of timesteps and the cost hamiltonian and possibly the number `q` of fourier parameters or the total annealing time `T`.

The parameters that are variable, are in the case of `AdiabaticTimestepsQAOAParameters`, the times at which to change the hamiltonian. In the case of `AlternatingOperatorsQAOAParameters` they are the `gammas_singles`, `gammas_pairs` and `betas` and for `FourierQAOAParameters` they are `v, u_singles, u_pairs`

In [21]:
# the constant parameters
ntimesteps = 3                # the number of timesteps

# put them into a nice tuple
hyperparameters = (hamiltonian, ntimesteps)

# the variable parameters --- in this case the timesteps
betas = [0.25, 0.5, 1.0]
gammas_singles = [1.0, 0.5, 0.3]
gammas_pairs = [1.0, 0.5, 0.3]
# and make them a tuple.
variable_parameters = (betas, gammas_singles, gammas_pairs)
manual_params = AlternatingOperatorsQAOAParameters(hyperparameters,
                                                   variable_parameters)
print(manual_params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.25, 0.5, 1.0]
	gammas_singles: [1.0, 0.5, 0.3]
	gammas_pairs: [1.0, 0.5, 0.3]



Now if we want to change this parameters, we can do it again by getting the flat list of all of them with `manual_params.raw()`, changing them and putting them back with `manual_params.update()`. But this requires, that we know where in the long list of parameters the one is. It is much easier, to just change change them manually:

In [22]:
manual_params.betas, manual_params.gammas_singles, manual_params.gammas_pairs =\
    betas, gammas_singles, gammas_pairs
print(manual_params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.25, 0.5, 1.0]
	gammas_singles: [1.0, 0.5, 0.3]
	gammas_pairs: [1.0, 0.5, 0.3]



And if we want to change only one value in the parameters, we can do this too:

In [23]:
manual_params.betas[0] = np.pi
print(manual_params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.5, 0.3]
	gammas_pairs: [1.0, 0.5, 0.3]



We see, that the `betas`, `gammas_singles` and `gammas_pairs` have changed, while the constant parameters `register`, `qubits_singles` and `qubits_pairs` didn't change. We can also see, what happened to the final coefficients, that are passed to `qaoa._qaoa_annealing_program`:

In [9]:
print("\n betas:\n", params.betas)
print("\n gammas_singles:\n", params.gammas_singles)
print("\n gammas_pairs:\n", params.gammas_pairs)


 betas:
 [[0.0, 0.0, 0.0], [0.125, 0.125, 0.125]]

 gammas_singles:
 [[-0.0625], [-0.1875]]

 gammas_pairs:
 [[0.0875, 0.15], [0.26249999999999996, 0.44999999999999996]]


Let us check once again, that the rotation angles got updated accordingly:

In [24]:
print("\n x_rotation_angles:\n", manual_params.x_rotation_angles)
print("\n z_rotation_angles:\n", manual_params.z_rotation_angles)
print("\n zz_rotation_angles:\n", manual_params.zz_rotation_angles)


 x_rotation_angles:
 [[3.141592653589793, 3.141592653589793, 3.141592653589793], [0.5, 0.5, 0.5], [1.0, 1.0, 1.0]]

 z_rotation_angles:
 [[-0.5], [-0.25], [-0.15]]

 zz_rotation_angles:
 [[0.7, 1.2], [0.35, 0.6], [0.21, 0.36]]


## Iterating over parameter ranges
Often enough, we want to see what happens if we keep all parameters except for one constant. Using `QAOAParameterIterator` gives a convenient way of doing that.

Take any object of subtype of `AbstractQAOAParamters`, e.g. the `manual_params` that we just created, and have a look what the variable parameters are, that can be changed by printing it:

In [25]:
print(manual_params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.5, 0.3]
	gammas_pairs: [1.0, 0.5, 0.3]



As we see, we could for example take one of the `gammas_singles` to iterate over. How about the second one `gammas_singles[1]`:

In [26]:
the_range = np.arange(0, 1, 0.3334)
the_parameter = "gammas_singles[1]"
iterator= QAOAParameterIterator(manual_params, the_parameter, the_range)
for p in iterator:
    print(p)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.0, 0.3]
	gammas_pairs: [1.0, 0.5, 0.3]

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.3334, 0.3]
	gammas_pairs: [1.0, 0.5, 0.3]

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.6668, 0.3]
	gammas_pairs: [1.0, 0.5, 0.3]



And we see, what we expected: In the for loop all parameters stay constant except for `gammas_singles[1]` which went up in steps of thirds. Of course, we can also nest these iterators, to sweep two-dimensional paramter landscapes:

In [27]:
range1 = np.arange(0,1,0.5)
range2 = np.arange(100,101,0.5)
for params in QAOAParameterIterator(manual_params, "gammas_singles[1]", range1):
    for p in QAOAParameterIterator(params, "gammas_pairs[0]", range2):
        print(p)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.0, 0.3]
	gammas_pairs: [100.0, 0.5, 0.3]

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.0, 0.3]
	gammas_pairs: [100.5, 0.5, 0.3]

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.5, 0.3]
	gammas_pairs: [100.0, 0.5, 0.3]

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.141592653589793, 0.5, 1.0]
	gammas_singles: [1.0, 0.5, 0.3]
	gammas_pairs: [100.5, 0.5, 0.3]

